# Scrape OHL Roster Data
To evaluate correlation of metrics and performance, we need to scrape roster/statistics data from OHL website.

In [132]:
from bs4 import BeautifulSoup
import pandas as pd
import pprint
import json
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pickle
import os

In [33]:
# Prepare webdriver
driver = webdriver.Chrome('/Users/williamshaw/Desktop/chromedriver')

In [41]:
# Access website via webdriver
driver.get('https://ontariohockeyleague.com/stats/team_players/68/2')

In [127]:
# Loop through each team, store player stats using web scrape
ohl_data = pd.DataFrame()

for i in range(1,21):
    # Go to webpage for specific team (team id's range from 1-20)
    print('mining team id: ' + str(i))
    driver.get('https://ontariohockeyleague.com/stats/team_players/68/' + str(i))
    time.sleep(1.5) # wait for webpage to load
    
    # First, get a list of players on a team
    players = driver.find_elements_by_xpath('//td[@class="table__td table__td-- text-col table__td-name"]')
    players_list = []
    for i in range(len(players)):
        players_list.append(players[i].text)
    print('players: ' + str(len(players_list)))
    
    # Second, get a list of team abreviations
    teams_list = []
    team_name = driver.find_elements_by_xpath('//td[@class="table__td table__td-- text-col"]')
    for i in range(len(team_name)):
        teams_list.append(team_name[i].text)
    print('team abbrev: ' + str(len(players_list)))
    
    # Third, get a list of all players stats
    stats_list = []
    stats = driver.find_elements_by_xpath('//td[@class="table__td"]')
    for i in range(len(stats)):
        stats_list.append(stats[i].text)
    print('num stats counted: ' + str(len(stats_list)))
    
    # Next, convert stats_list to 2d array
    stats_matrix = []
    count = 0
    tmp = []
    for i in range(0, len(stats_list)):
        tmp.append(stats_list[i])
        count += 1
        if count == 28:
            stats_matrix.append(tmp)
            tmp = []
            count = 0
    print('len of stats_matrix: ' + str(len(stats_matrix)))
    
    # Finally, convert roster data into a dataframe an append to ohl_data
    df_tmp = pd.DataFrame(stats_matrix)
    df_tmp['name'] = players_list
    df_tmp['team'] = teams_list
    ohl_data = pd.concat([ohl_data, df_tmp], ignore_index=True, sort=False)
    print()
    

mining team id: 1
players: 27
team abbrev: 27
num stats counted: 756
len of stats_matrix: 27

mining team id: 2
players: 35
team abbrev: 35
num stats counted: 980
len of stats_matrix: 35

mining team id: 3
players: 0
team abbrev: 0
num stats counted: 0
len of stats_matrix: 0

mining team id: 4
players: 36
team abbrev: 36
num stats counted: 1008
len of stats_matrix: 36

mining team id: 5
players: 32
team abbrev: 32
num stats counted: 896
len of stats_matrix: 32

mining team id: 6
players: 35
team abbrev: 35
num stats counted: 980
len of stats_matrix: 35

mining team id: 7
players: 46
team abbrev: 46
num stats counted: 1288
len of stats_matrix: 46

mining team id: 8
players: 31
team abbrev: 31
num stats counted: 868
len of stats_matrix: 31

mining team id: 9
players: 32
team abbrev: 32
num stats counted: 896
len of stats_matrix: 32

mining team id: 10
players: 31
team abbrev: 31
num stats counted: 868
len of stats_matrix: 31

mining team id: 11
players: 27
team abbrev: 27
num stats count

In [134]:
# Print out dataframe
ohl_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 631 entries, 0 to 630
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       631 non-null    object
 1   1       631 non-null    object
 2   2       631 non-null    object
 3   3       631 non-null    object
 4   4       631 non-null    object
 5   5       631 non-null    object
 6   6       631 non-null    object
 7   7       631 non-null    object
 8   8       631 non-null    object
 9   9       631 non-null    object
 10  10      631 non-null    object
 11  11      631 non-null    object
 12  12      631 non-null    object
 13  13      631 non-null    object
 14  14      631 non-null    object
 15  15      631 non-null    object
 16  16      631 non-null    object
 17  17      631 non-null    object
 18  18      631 non-null    object
 19  19      631 non-null    object
 20  20      631 non-null    object
 21  21      631 non-null    object
 22  22      631 non-null    ob

In [151]:
pd.options.display.max_columns = None
ohl_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,name,team
0,1,RW,34,57,44,54,98,-7,28,16,25,1,2,288,2,0,8,6,4,0,4,0,0.0,22,4,18.2,1.72,0.49,"Kaliyev, Arthur",HAM
1,2,C,21,27,22,34,56,12,30,3,12,3,1,94,0,0,4,2,2,0,2,0,0.0,581,251,43.2,2.07,1.11,"Jenik, Jan",HAM
2,3,C,84,57,25,22,47,-29,23,8,9,0,0,154,2,1,3,0,4,3,4,3,75.0,156,65,41.7,0.82,0.40,"Bertuzzi, Tag",HAM
3,4,C,9,59,23,22,45,-33,18,9,10,0,0,148,2,0,3,0,1,0,1,0,0.0,908,441,48.6,0.76,0.31,"Morrison, Logan",HAM
4,5,D,44,60,6,35,41,-20,59,3,19,0,1,153,1,0,2,0,0,0,0,0,0.0,0,0,0.0,0.68,0.98,"Staios, Nathan",HAM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626,,,17,10,0,1,1,-13,2,0,0,0,0,22,0,0,0,0,0,0,0,0,0.0,136,53,39.0,0.10,0.20,NIAG,NIAG
627,28,G,37,24,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0.0,0.00,0.21,"Sbaraglia, Christian",NIAG
628,29,G,30,39,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0.0,0.00,0.05,"MacLean, Andrew",NIAG
629,,,35,19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0.0,0.00,0.00,OSH,OSH


In [152]:
# Convert Column Names
df = ohl_data.copy()   # make a copy of the dataframe so we can safely edit things and go back if we mess up
df.columns = ['rank', 
              'pos',
              'jersey_number',
              'gp',
              'g',
              'a',
              'pts',
              'plus_minus',
              'pim',
              'ppg',
              'ppa',
              'shg',
              'sha',
              'sog',
              'gwg',
              'otg',
              'first_goals',
              'insurance_goals',
              'shootout_gp',
              'shootout_goals',
              'shootout_attempts',
              'shootout_winning_goals',
              'shootout_percent',
              'fo_attempts',
              'fo_wins',
              'fo_percent',
              'pts_per_game',
              'pim_per_game',
              'name',
              'team'
             ]

In [164]:
# Convert datatypes to numeric where possible
df[['rank', 
      'gp',
      'g',
      'a',
      'pts',
      'plus_minus',
      'pim',
      'ppg',
      'ppa',
      'shg',
      'sha',
      'sog',
      'gwg',
      'otg',
      'first_goals',
      'insurance_goals',
      'shootout_gp',
      'shootout_goals',
      'shootout_attempts',
      'shootout_winning_goals',
      'shootout_percent',
      'fo_attempts',
      'fo_wins',
      'fo_percent',
      'pts_per_game',
      'pim_per_game']] = df[['rank', 
                              'gp',
                              'g',
                              'a',
                              'pts',
                              'plus_minus',
                              'pim',
                              'ppg',
                              'ppa',
                              'shg',
                              'sha',
                              'sog',
                              'gwg',
                              'otg',
                              'first_goals',
                              'insurance_goals',
                              'shootout_gp',
                              'shootout_goals',
                              'shootout_attempts',
                              'shootout_winning_goals',
                              'shootout_percent',
                              'fo_attempts',
                              'fo_wins',
                              'fo_percent',
                              'pts_per_game',
                              'pim_per_game']].apply(pd.to_numeric)

In [166]:
# Get rid of columns where there is no rank or position 
# (these columns exist because a player was traded mid-season)
df = df.dropna(how='any',axis=0)

In [167]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 540 entries, 0 to 628
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   rank                    540 non-null    float64
 1   pos                     540 non-null    object 
 2   jersey_number           540 non-null    object 
 3   gp                      540 non-null    int64  
 4   g                       540 non-null    int64  
 5   a                       540 non-null    int64  
 6   pts                     540 non-null    int64  
 7   plus_minus              540 non-null    int64  
 8   pim                     540 non-null    int64  
 9   ppg                     540 non-null    int64  
 10  ppa                     540 non-null    int64  
 11  shg                     540 non-null    int64  
 12  sha                     540 non-null    int64  
 13  sog                     540 non-null    int64  
 14  gwg                     540 non-null    in

In [171]:
df.sort_values('g', ascending=False)

,rank,pos,jersey_number,gp,g,a,pts,plus_minus,pim,ppg,ppa,shg,sha,sog,gwg,otg,first_goals,insurance_goals,shootout_gp,shootout_goals,shootout_attempts,shootout_winning_goals,shootout_percent,fo_attempts,fo_wins,fo_percent,pts_per_game,pim_per_game,name,team
130,1.0,LW,16,46,55,31,86,29,40,13,15,8,1,255,10,0,4,7,1,1,1,0,100.0,25,10,40.0,1.87,0.87,"Robertson, Nick",PBO
104,7.0,LW,22,62,52,37,89,48,32,15,16,3,2,240,7,2,4,8,3,1,3,0,33.3,73,34,46.6,1.44,0.52,"Quinn, Jack",OTT
403,1.0,C,11,52,47,55,102,32,26,16,26,2,0,283,8,1,4,7,1,0,1,0,0.0,880,493,56.0,1.96,0.50,"McMichael, Connor",LDN
246,3.0,LW,17,63,45,51,96,24,66,11,20,1,0,273,4,1,7,3,8,4,8,1,50.0,17,6,35.3,1.52,1.05,"Gogolev, Pavel",GUE
68,5.0,C,13,62,45,38,83,1,20,14,19,2,0,298,7,1,5,1,3,0,3,0,0.0,799,461,57.7,1.34,0.32,"Neumann, Brett",OSH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,24.0,D,4,8,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0.0,0,0,0.0,0.00,0.00,"Kavanagh, Josh",PBO
156,23.0,RW,18,4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,1,1,100.0,0.00,0.00,"Patey, Cole",PBO
155,22.0,G,30,17,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0.0,0.06,0.00,"Austin, Tye",PBO
153,20.0,D,24,14,0,2,2,3,4,0,0,0,0,6,0,0,0,0,0,0,0,0,0.0,0,0,0.0,0.14,0.29,"Guryev, Artem",PBO


In [169]:
# Store dataframe in pickle file
path = os.path.abspath(os.path.join(os.getcwd(), '..'))
path = os.path.abspath(os.path.join(path, 'data/ohl_data_2019_20.pkl'))
print(path)

df.to_pickle(path)

/Users/williamshaw/Projects/NHL/Big-Data-Cup-2021/data/ohl_data_2019_20.pkl
